<a href="https://colab.research.google.com/github/Aeroknight786/Kalman-Filter/blob/main/MIRR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime as datetime
import pandas as pd
import math
import numpy as np

In [ ]:
df_cashflows = pd.read_excel("Cashflows_8_7_21_final.xlsx")
df_cashflows.set_index('Date', inplace = True)  
print(df_cashflows.tail(11))

              Cash Flows
Date                    
2020-09-21 -5.893195e+09
2020-09-22 -2.002457e+10
2020-09-23 -3.978818e+10
2020-09-24 -8.028312e+09
2020-09-25 -2.087172e+10
2020-09-26  0.000000e+00
2020-09-27  0.000000e+00
2020-09-28  2.335230e+08
2020-09-29 -1.383740e+10
2020-09-30 -6.763029e+09
2020-10-01 -3.465811e+06


In [ ]:
df_merged = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/merged_8_7_21_final.csv",error_bad_lines=False))
df_merged

,Unnamed: 0,Unnamed: 0.1,SCRIP_NAME,ISIN,TR_DATE,TR_TYPE(*),VALUE (in Rs),RATE,QUANTITY
0,0,8,Hindustan Lever Ltd.,INE030A01027,12/31/2002,4,998030.00,181.4600,5500.0
1,1,9,Tata Iron and Steel Co.Ltd,INE081A01012,12/31/2002,4,1637350.00,148.8500,11000.0
2,2,10,Infosys Technologies Ltd.,INE009A01021,12/31/2002,4,2378725.00,4757.4500,500.0
3,3,12,Hero Honda Motors Limited,INE158A01026,12/31/2002,1,6206090.00,269.8300,23000.0
4,4,15,"Reliance Industries Ltd.,",INE002A01018,12/31/2002,4,22966020.00,298.2600,77000.0
...,...,...,...,...,...,...,...,...,...
10556686,10556686,8940134,ADANI ENTERPRISES LIMITED,INE423A01024,30-09-2020,1,7428062.88,297.6583,24955.0
10556687,10556687,8940136,RELIANCE IND LTD DM,INE002A01018,30-09-2020,4,145104.70,2232.3800,65.0
10556688,10556688,8940138,RELIANCE IND LTD DM,INE002A01018,30-09-2020,4,748678.87,2234.8623,335.0
10556689,10556689,8940143,LAURUS LABS LTD DM2,INE947Q01028,30-09-2020,4,15617855.28,292.4692,53400.0


In [ ]:
dates = df_merged.TR_DATE.unique()
print((dates))
date_patterns =  ["%d-%m-%Y","%m/%d/%Y"]
def date_convert(date):
  for pattern in date_patterns:
    try:
      formatted_date = datetime.datetime.strptime(date,pattern)
      format = pattern    
      break
    except:
      pass
  return formatted_date

date_list = []

for date in dates:
  formatted_date = date_convert(date)
  date_list.append(formatted_date)

date_list.sort() 
date_list = date_list[1:]   
print(date_list)

['12/31/2002' '12/30/2002' '1/1/2003' ... '28-09-2020' '29-09-2020'
 '30-09-2020']
[datetime.datetime(2002, 12, 31, 0, 0), datetime.datetime(2003, 1, 1, 0, 0), datetime.datetime(2003, 1, 2, 0, 0), datetime.datetime(2003, 1, 3, 0, 0), datetime.datetime(2003, 1, 6, 0, 0), datetime.datetime(2003, 1, 7, 0, 0), datetime.datetime(2003, 1, 8, 0, 0), datetime.datetime(2003, 1, 9, 0, 0), datetime.datetime(2003, 1, 10, 0, 0), datetime.datetime(2003, 1, 13, 0, 0), datetime.datetime(2003, 1, 14, 0, 0), datetime.datetime(2003, 1, 15, 0, 0), datetime.datetime(2003, 1, 16, 0, 0), datetime.datetime(2003, 1, 17, 0, 0), datetime.datetime(2003, 1, 20, 0, 0), datetime.datetime(2003, 1, 21, 0, 0), datetime.datetime(2003, 1, 22, 0, 0), datetime.datetime(2003, 1, 23, 0, 0), datetime.datetime(2003, 1, 24, 0, 0), datetime.datetime(2003, 1, 27, 0, 0), datetime.datetime(2003, 1, 28, 0, 0), datetime.datetime(2003, 1, 29, 0, 0), datetime.datetime(2003, 1, 30, 0, 0), datetime.datetime(2003, 1, 31, 0, 0), datetime.d

In [ ]:
df_price_data = pd.DataFrame(pd.read_excel("/content/nifty 500 price data.xlsx"))  #Reading the Price data indexed by ISIN Codes
df_price_data.set_index('Date', inplace = True)                              #Setting the index of the dataframe as the date
df_price_data = df_price_data.fillna(0,axis = 1)                             #Filling absent price data fields with zero.
print(df_price_data.head(20))

            INE470A01017  INE117A01022  ...  INE520A01027  INE768C01010
Date                                    ...                            
2002-12-31        302.55         49.73  ...         10.18          0.00
2003-01-01        303.35         49.98  ...         10.09          0.00
2003-01-02        303.10         50.07  ...          9.86          0.00
2003-01-03        302.95         50.92  ...          9.79          0.00
2003-01-06        313.75         52.71  ...          9.54          0.00
2003-01-07        315.55         52.67  ...          9.56          0.00
2003-01-08        306.70         53.67  ...          9.72          0.00
2003-01-09        315.90         53.88  ...          9.72          0.00
2003-01-10        315.00         53.09  ...          9.39          0.00
2020-09-25      18222.15        852.35  ...        198.35       1863.85
2020-09-28      18375.75        861.70  ...        194.45       1839.75
2020-09-29      18404.00        854.45  ...        190.10       

In [ ]:
df_holdings = pd.DataFrame(pd.read_excel("/content/ISINHoldings_08_07_21_final.xlsx")) 
df_holdings.set_index('Date', inplace = True)
print(df_holdings.head(20))

            INE470A01017  INE117A01022  ...  INE520A01027  INE768C01010
Date                                    ...                            
2003-01-01          2566      19290902  ...       2775575       1126935
2003-01-02          2566      19290902  ...       2775575       1126935
2003-01-03          2566      19290902  ...       2775575       1126935
2003-01-04          2566      19290902  ...       2775575       1126935
2003-01-05          2566      19290902  ...       2775575       1126935
2003-01-06          2566      19290902  ...       2775575       1126935
2003-01-07          2566      19290902  ...       2775575       1126935
2003-01-08          2566      19290902  ...       2775575       1126935
2003-01-09          2566      19290902  ...       2775575       1126935
2003-01-10          2566      19290902  ...       2775575       1126935
2003-01-11          2566      19290902  ...       2775575       1126935
2003-01-12          2566      19290902  ...       2775575       

In [ ]:
np_prices_day_0 = np.array(df_price_data.loc[date_list[1]])
np_opening_holdings = np.array(df_holdings.loc[date_list[1]])
np_closing_holdings = np.array(df_holdings.loc[date_list[-1]])
np_prices_day_n  = np.array(df_price_data.loc[date_list[-1]])

print(date_list[1])
entry_value = -1 * np.dot(np_opening_holdings,np_prices_day_0)

print(date_list[-1])
exit_value = np.dot(np_closing_holdings,np_prices_day_n )
print(str(entry_value) + " " + str(exit_value))


2003-01-01 00:00:00
2020-09-30 00:00:00
-140803587554.15002 6465389541851.08


In [ ]:
print(len(date_list))
def cumulative_profitability(df, present_value, future_value):
  
  date_counter = 0
  rate = 0.08
  for date in date_list:
    cash_flow = df_cashflows._get_value(date, 'Cash Flows')
    #print(cash_flow)
    if (cash_flow < 0):
      #print(date_counter)
      present_value += cash_flow * math.exp(-1 * rate * (date_counter/365))
    else:
      future_value += cash_flow * math.exp(rate * (17 - (date_counter/365)))
    date_counter += 1
 
  print("Present Value: " + str(present_value) + " Future Value: " + str(future_value ))
  print((math.log(future_value) - math.log(abs(present_value)))/17)
  print(date_counter)
cumulative_profitability(df_cashflows, entry_value, exit_value)

4423
Present Value: -4390721915143.9966 Future Value: 31047599355950.008
0.11506046064891962
4423


In [11]:
def nifty_sim(entry_price, exit_price):
  print((math.log(exit_price) - math.log(entry_price))/12)
nifty_sim(2500,13000)

0.13738821879894844


In [13]:
print(math.log(14000/1100)/17)

0.1496321852829961
